### Tokenize and Lemmatize inputs

Source: lecture notebooks + https://gist.github.com/4OH4/f727af7dfc0e6bb0f26d2ea41d89ee55

In [122]:
import pandas as pd
import json
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import pickle

import spacy
import numpy as np

In [ ]:
train = pd.read_csv('data/train.csv')
valid = pd.read_csv('data/valid.csv')
test = pd.read_csv('data/valid.csv')

In [155]:
def lemmatize_embeddings(dat=object):
    
    dat = dat.fillna('')
    dat['input'] = dat['title']+dat['abstract']
    dat['key'] = 'PMID:'+dat['pmid'].astype(str)
    
    dat_dict = {dat['key'][i]: {'input': dat['input'][i], 
                                'label': dat['label'][i]}
                for i in range(len(dat))}
    
    load_model = spacy.load('en_core_web_md', disable = ['parser','ner'])
    
    # Lemmatize and add to dictionary
    for i in dat_dict:
        doc = load_model(dat_dict[i]['input'])
        dat_dict[i]['lemmas'] = [token.lemma_ for token in doc if len(token.lemma_) > 1]
    
    # Generate word embeddings for each lemma
    nlp = spacy.load('en_core_web_md')
    
    for i in dat_dict:
        embeddings = {j: nlp.vocab[j].vector for j in dat_dict[i]['lemmas']}
        sentenceMatrix = np.array([embeddings[i] for i in embeddings])
        dat_dict[i]['embeddings'] = sentenceMatrix
    
    return dat_dict

In [157]:
%%time

train_dict = lemmatize_embeddings(train)
valid_dict = lemmatize_embeddings(valid)
test_dict = lemmatize_embeddings(test)

CPU times: user 3min 33s, sys: 28.5 s, total: 4min 1s
Wall time: 4min 17s


In [159]:
# pickle.dump(train_dict, open('data/train.pkl', 'wb'))
# pickle.dump(valid_dict, open('data/valid.pkl', 'wb'))
# pickle.dump(test_dict, open('data/test.pkl', 'wb'))